In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from IPython.display import display

from datetime import datetime, timedelta

In [2]:
# Pega o usuario e senha 
with open('arquivo','r') as arquivo:
    conteudo = arquivo.readlines()

usuario, senha = [conteudo[0].strip(),conteudo[1].strip()] 

print(usuario, senha)

clerisson clerissonshalom


In [3]:
class WOP:
    """Cria um objeto WOP. 
    1) [automático] Ler a lista de grupos 
    1) [automático] Iniciar o Selenium com WOP.inicia_selenium()
    2) [automático] Fazer o login com WOP.login(usuario,senha)
    """
    def __init__(self, path_grupos_csv = './grupos.csv', path_driver="./geckodriver", usuario=usuario, senha=senha):
        self.path_grupos_csv = path_grupos_csv
        self.path_driver = path_driver
        self.usuario = usuario
        self.senha = senha
        self.grupos_df = self.get_grupos_df()
        self.driver = self.inicializa_selenium()
        self.faz_login()

    def inicializa_selenium(self):
        """Inicializa o selenium
        return: selenium.webdriver object
        """
        geckodriver_path = self.path_driver
        options = Options()
        #options.add_argument('--headless')  # Se quiser rodar o Firefox em modo headless
        service = FirefoxService(executable_path=geckodriver_path)
        driver = webdriver.Firefox(service=service, options=options)
        print("| Selenium Iniciado!")
        return driver

    def faz_login(self):
        print("| Fazendo Login")

        driver = self.driver
        pagina_de_login='https://wop.obra.shalom.tec.br/login'
        driver.get(pagina_de_login)

        username_field = driver.find_element(By.NAME, 'username')  # Use o nome correto do campo de usuário
        password_field = driver.find_element(By.NAME, 'password')  # Use o nome correto do campo de senha

        username_field.clear()
        password_field.clear()
        username_field.send_keys(usuario)
        password_field.send_keys(senha)

        login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')  # Use o seletor correto para o botão de login
        login_button.click()

        print("|-- Login Ok")
        display(self.grupos_df)
        #return 1

    def get_grupos_df(self):
        """Carrega a lista e informações do grupos a partir de um csv

        Returns:
            pd.Dataframe: Dataframe com os grupos
        """        
        grupos_df = pd.read_csv(self.path_grupos_csv).sort_values(by=["CEV", "nome"])
        return grupos_df

    # Método para pegar as ovelhas do grupo
    def get_ovelhas(self, grupo_id, to_df=True, exporta=False, exporta_path = './grupos'):
        """Entra no link de um determinado grupo e cria um list de list contendo 
    as ovelhasjá cadastradas nesse grupo de oração.
    Args:
        to_df: booleano que indica se o valor deve ser retornado já em um pandas df
    Returns:
        um list de lists no formato [[...], [...], [...]] a menos que
        seja indicado to_df
    """ 
        id_do_grupo = grupo_id
        grupos_df = self.grupos_df

        grupo_selecionado = grupos_df.loc[grupos_df["id"]==grupo_id]

        if not grupo_selecionado.empty:
            nome_grupo = grupo_selecionado.iloc[0]['nome']
            CEV_grupo = grupo_selecionado.iloc[0]['CEV']
        else:
            nome_grupo = None
            print(f"Grupo com id {id_do_grupo} não encontrado.")
            raise "Grupo Não encontrado"
        
        exporta_str = f"{exporta_path}/{CEV_grupo}_{id_do_grupo}_{nome_grupo}.csv"

        print(f"| Obtendo a lista de membros do grupo {nome_grupo}")

        driver = self.driver
        driver.get(f"https://wop.obra.shalom.tec.br/secured/local/grupodeoracao/{ id_do_grupo }/gerenciar")
        

        try:
            # Espere até que o elemento esteja visível (aguarde até 10 segundos)
            xpath= "/html/body/app-root/grupooracao-local-manage/secured/body/div/span/div/section[2]/div/div[4]/div/div[3]/div/div/table-component/div/div[2]/table/tbody"
            tabela = WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.XPATH, xpath))
            )
        except:
            print("|!- O elemento não foi carregado a tempo")
            df = pd.DataFrame( columns=["user id","Nome", "Data de ingresso", "Status"])
            df.to_csv(exporta_str, index=False)
            return df

        linhas = tabela.find_elements(By.TAG_NAME, 'tr')

        # Cria um list com as ovelhas encontradas na tabela
        membros_list =[]
        for linha in linhas:
            colunas = linha.find_elements(By.TAG_NAME, 'td')
            dados = [coluna.text for coluna in colunas if coluna.text != '']
            membros_list.append(dados)

        membros_df = pd.DataFrame(membros_list, columns=["user id","Nome", "Data de ingresso", "Status"])
        membros_df = membros_df.sort_values(by='Nome')

        if exporta:
            membros_df.to_csv(exporta_str, index=False)
            print("|-- Exportado para"+exporta_str)
            

        # Decide qual será a saída
        if to_df:
            return membros_df
        else:
            return membros_list

    def get_frequencia(self, id):
        """Obtem a frequência do grupo dado pelo id

        Args:
            id (_type_): _description_

        Returns:
            _type_: _description_
        """        
        id_do_grupo = id
        
        driver = self.driver

        driver.get(f" https://wop.obra.shalom.tec.br/secured/local/grupodeoracao/{ id_do_grupo }/presenca")

        # Pega as frequencias
        try:
            # Espere até que o elemento esteja visível (aguarde até 10 segundos)
            #xpath= "/html/body/app-root/presenca-grupooracao-local/secured/body/div/span/div/section[2]/div/div/div/div/div[2]/div/div[2]/table"
            css_selector = ".table > tbody:nth-child(2)"
            tabela = WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR,css_selector))#((By.XPATH, xpath))
            )
        except:
            print("|!- Frequência não encontrada")
            return []
        
        frequencias = tabela.text.splitlines()
        return frequencias

    def id_to_nome(self,grupo_id):
        """Helper Function que pega o nome a partir do id do grupo

        Args:
            grupo_id (_type_): _description_
        Returns:
            _type_: _description_
        """    
        # Inicializa o nome do grupo com base no id
        id_do_grupo = grupo_id
        grupo_selecionado = self.grupos_df[self.grupos_df['id'] == id_do_grupo]
        if not grupo_selecionado.empty:
            nome = grupo_selecionado.iloc[0]['nome']
        else:
            nome = None
            print(f"Grupo com id {self.id} não encontrado.")
        return nome
    

WOP = WOP()

| Selenium Iniciado!
| Fazendo Login
|-- Login Ok


,id,nome,fase,publico,CEV
19,964,Fiat,Kerygma,Adultos,Ananindeua
0,324,Israel,fase,???,Ananindeua
2,326,Kadosh,Metanoia,Adultos,Ananindeua
3,327,Mãe da Misericórdia,Kerygma,Jovens,Ananindeua
4,725,Pantocrator,Kerygma,Adultos,Ananindeua
5,727,Santo Agostinho,Kerygma,Jovens,Ananindeua
1,325,São Tomé,Kerygma,Adultos,Ananindeua
17,1148,Bom Pastor,Kerygma,Adultos,São Brás
10,301,Fiat,Kerygma,Adultos,São Brás
12,484,Filhas da Imaculada,Koinonia,Adultos,São Brás


In [ ]:
grupo = WOP.get_ovelhas(921)
grupo.head()

In [ ]:
for i,grupo_id in enumerate(WOP.get_grupos_df()['id']):
    print(f"[{i+1}/{WOP.grupos_df["id"].size}] ")
    WOP.get_ovelhas(grupo_id,exporta=True)


In [22]:
freq = WOP.get_frequencia(921)
freq

['12/09/2024', '05/09/2024']

In [ ]:
driver = WOP.driver
tabelas = driver.find_elements(By.CSS_SELECTOR,".table")[0]

texto = tabelas.text
texto.splitlines()

10

In [19]:
def gera_datas(data_inicio="01/01/2024"):
    """Gera todas as datas entre uma data de inicio e a data atual

    Args:
        data_inicio (str, optional): _description_. Defaults to "01/01/2024".
    """    
    data_inicio_sting = data_inicio.split("/")
    print(data_inicio_sting)

    data_fim = datetime.today().date().strftime("%d/%m/%Y")
    data_fim_str = data_fim.split("/")
    print(data_fim_str)
    
    dia_i, mes_i, ano_i = data_inicio_sting
    dia_f, mes_f, ano_f = data_inicio_sting

    quintas = []
    quartas = []
    sabados = []
    tercas = []
    segundas = []

    for ano in range(int(ano_i),int(ano_f)+1):
        for mes in range(int(mes_i),int(mes_f)+1):
            for dia in range(int(dia_i),int(dia_f)+1):
                data = f"{dia}/{mes}/{ano}"
                print(data)
                try:
                    data_dt = datetime.strptime(data,"%d/%m/%Y")
                except:
                    continue

                if data_dt.weekday() == 0:
                    segundas.append(data_dt.date())

                if data_dt.weekday() == 1:
                    tercas.append(data_dt.date())

                if data_dt.weekday() == 2:
                    quartas.append(data_dt.date())

                if data_dt.weekday() == 3:
                    quintas.append(data_dt.date())

                if data_dt.weekday() == 5:
                    sabados.append(data_dt.date())
    print(quartas)
gera_datas()

['01', '01', '2024']
['18', '09', '2024']
1/1/2024
[]


In [ ]:
Grupo = pd.DataFrame(membros_list, columns=["user id","Nome", "Data de ingresso", "Status"])
Grupo = Grupo.sort_values(by='Nome')
Grupo

In [ ]:
Grupo.to_csv(path_to_export_csv,index=False)

In [ ]:
# Exporta todos os grupos
path_to_export_csv = "./grupos"
for grupo in grupos_df.itertuples():

    grupo_id = grupo.id
    grupo_nome = grupo.nome
    grupo_CEV = grupo.CEV
    grupo_nome = grupo_nome.replace(" ","_")

    texto  = f"{grupo_id}\t{grupo_nome}\t{grupo.CEV}"
    print(texto)

    membros_list = pega_membros_do_grupo(grupo_id,driver)

    if membros_list is None:
        print("=> Falhou")
    
    membros_df = pd.DataFrame(membros_list, columns=["user id","Nome", "Data de ingresso", "Status"])
    membros_df.sort_values(by='Nome')

    membros_df.to_csv(f"{path_to_export_csv}/{grupo_CEV}_{grupo_id}_{grupo_nome}.csv",index=False)


In [14]:
# Faechar o navegador
driver.quit()

In [ ]:
print("|Testando")
print("|-Testando")
print("|--Testando 2")